In [ ]:
%matplotlib widget
import os, sys, json
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import dates
import pickle as pkl
import ipywidgets as widget
from ipywidgets import interact
import ipydatetime
import datetime
import numpy as np
from datetime import datetime, timezone

def get_day_hour(date):
    if "T" in date:
        start = date.split('_')[0][:-1]
        stop = date.split('_')[1][:-1]
        start_day = datetime.strptime(start,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
        stop_day = datetime.strptime(stop,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
        time_interval = start_day+" -> "+stop_day
    else:
        time_interval = "no time cuts"
        
    
    return time_interval

In [ ]:
def get_day_hour(date):
    start = date.split('_')[0][:-1]
    stop = date.split('_')[1][:-1]
    start_day = datetime.strptime(start,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
    stop_day = datetime.strptime(stop,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
    
    return start_day+" -> "+stop_day

In [ ]:
exp         = 'l60'
period      = 'p01'
datatype    = 'phy'
foldername  = 'out'

pkl_name = exp + '-' + period

pkl_files = os.listdir(f'{foldername}/pkl-files/par-vs-time/')
pkl_files = [file for file in pkl_files if 'pkl' in file] 

runs = list(dict.fromkeys([file.split('-')[2] for file in pkl_files]))
print("Available runs: ", runs)

geds_list = [file for file in pkl_files if 'string' in file if 'pulser' not in file]      # geds: grouped by string
geds_list_with_date = [file for file in geds_list if 'Z_' in file]
spms_list = [file for file in pkl_files if 'string' not in file if 'pulser' not in file]  # spms: everything not grouped by string
puls_list = [file for file in pkl_files if 'pulser' in file]  # pulser: ch000

# geds
if geds_list != []:
    # configuration
    geds_map = list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in geds_list]))
    # parameters
    geds_par = list(dict.fromkeys([file.split('-')[-2] for file in geds_list]))
    geds_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in geds_list_with_date])))

    print("\ngeds map:", geds_map)
    print("geds par:", geds_par)
else: 
    geds_map = geds_par = geds_date = []
    print("\n-> NO data for geds were found")

geds_date_formatted = [get_day_hour(date) for date in geds_date]
geds_time_option = [(key, value) for key,value in zip(geds_date_formatted, geds_date)]
geds_time_option.append(("all", "all"))
print("Time intervals:", geds_time_option)

In [ ]:
# spms
if spms_list != []:
    # configuration
    spms_map  = list(dict.fromkeys([file.split('-')[-2].split('.')[0] for file in spms_list if 'OB' in file if 'gain' not in file]))
    spms_map  += list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in spms_list if 'IB' in file if 'gain' not in file]))  
    spms_map_gain  = list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in spms_list if 'gain' in file]))  
    spms_map_div = []
    for entry in spms_map:
        if entry=='top_OB' or entry=='bot_OB':
            spms_map_div.append(entry+'-1')
            spms_map_div.append(entry+'-2')
        else:
            spms_map_div.append(entry)
    # parameters
    spms_par  = list(dict.fromkeys([file.split('-')[-3] for file in spms_list if 'OB' in file if 'gain' not in file]))
    spms_par  += list(dict.fromkeys([file.split('-')[-2] for file in spms_list if 'IB' in file if 'gain' not in file]))
    spms_par_gain  = list(dict.fromkeys([file.split('-')[-2] for file in spms_list if 'gain' in file]))
    # date
    spms_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in spms_list])))
    for entry in spms_date:
        if "Z_" not in entry:
            spms_date.remove(entry)
    
    # remove duplicates
    spms_par = list(dict.fromkeys(spms_par))
    spms_map = list(dict.fromkeys(spms_par))
    
    print("\nspms map (div):", spms_map_div)
    if spms_map_gain!=[]: print("spms map (gain):", spms_map_gain)
    print("spms par:", spms_par_gain+spms_par)
else:    
    spms_map = spms_map_gain = spms_map_div = spms_par = spms_par_gain = spms_date = []
    print("\n-> NO data for spms were found")   
    
spms_date_formatted = [get_day_hour(date) for date in spms_date]
spms_time_option = [(key, value) for key,value in zip(spms_date_formatted, spms_date)]
if spms_date==[] and spms_time_option==[]:
    spms_time_option = [('all','no_time_cuts')]
    spms_date.append('no_time_cuts')
    
# pulser
if puls_list != []:
    # parameters
    puls_par = list(dict.fromkeys([file.split('-')[-2] for file in puls_list]))
    # date
    puls_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in puls_list])))
    for entry in puls_date:
        if "Z_" not in entry:
            puls_date.remove(entry)
    print("\npuls par:", puls_par)
else: 
    puls_par = puls_date = []
    print("\n-> NO data for puls were found")

puls_date_formatted = [get_day_hour(date) for date in puls_date]
puls_time_option = [(key, value) for key,value in zip(puls_date_formatted, puls_date)]
if puls_date==[] and puls_time_option==[]:
    puls_time_option = [('all','no_time_cuts')]
    puls_date.append('no_time_cuts')

In [ ]:
if geds_date!=[]:
    # select widget for selecting the date (for geds) 
    geds_time_select = widget.Select(
        options=geds_time_option,
#        value=geds_time_option[0],
        # rows=10,
        description='Time:',
        disabled=False,
        layout={'width': 'max-content'}
)
if spms_date!=[]:
    # select widget for selecting the date (for spms)
    spms_time_select = widget.Select(
        options=spms_time_option,
        value=spms_date[0],
        # rows=10,
        description='Time:',
        disabled=False,
        layout={'width': 'max-content'}
)
if puls_date!=[]:
    # select widget for selecting the date (for puls)
    puls_time_select = widget.Select(
        options=puls_time_option,
        value=puls_date[0],
        # rows=10,
        description='Time:',
        disabled=False,
        layout={'width': 'max-content'}
)
    
# slider for z-axis range
zrange_slider = widget.FloatRangeSlider(
    #min=-50000, max=50000, step=1000, value=[-500,500], 
    min=-1000000, max=1000000, step=1, value=[-1000000,1000000], 
    readout_format='d', 
    description='z-axis range',
    width='1000px'
)

# slider for 3D angle view
elevation_slider = widget.IntSlider(
    min=-360, max=360, step=5, value=20,
    description='Elevation angle [°]:',
    style={'description_width': 'initial'}
)

azimuth_slider = widget.IntSlider(
    min=-360, max=360, step=5, value=-60,
    description='Azimuth angle [°]:',
    style={'description_width': 'initial'}
)

# tab for selecting parameters to plot (for geds)
run_button = widget.ToggleButtons(
    options= runs,
    description='Run:',
    disabled=False,
    button_style='',
    tooltips=[]
)

# tab for selecting parameters to plot (for geds)
par_geds_buttons = widget.ToggleButtons(
    options= geds_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)
# tab for selecting parameters to plot (for spms)
par_spms_buttons = widget.ToggleButtons(
    options= spms_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)
# tab for selecting parameters to plot (for puls)
par_puls_buttons = widget.ToggleButtons(
    options= puls_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)

# par-vs-time
geds_map_buttons = widget.ToggleButtons(
    options= geds_map,
    description='String #:',
    disabled=False,
    button_style='',
    tooltips=[],
) 
spms_map_buttons = widget.ToggleButtons(
    options= spms_map_div, 
    description='Position:',
    disabled=False,
    button_style='',
    tooltips=[],
) 

# gain
spms_map_buttons_gain = widget.ToggleButtons(
    options= spms_map_gain,
    description='Position:',
    disabled=False,
    button_style='',
    tooltips=[],
) 

# run widget
run_button = widget.ToggleButtons(
    options= runs,
    description='Run:',
    disabled=False,
    button_style='',
    tooltips=[]
)

In [ ]:
def plot_geds_3D(zrange, elevation, azimuth, time_select, parameter, string, run):
    if time_select == 'no_time_cuts':
        ax = pkl.load(open(f'{foldername}/pkl-files/par-vs-time/{pkl_name}-{run}-{datatype}-{parameter}-{string}.pkl', 'rb'))
    else:
        ax = pkl.load(open(f'{foldername}/pkl-files/par-vs-time/{pkl_name}-{run}-{datatype}-{time_select}-{parameter}-{string}.pkl', 'rb'))
    ax.set_zlim3d(zrange)
    ax.figure.set_size_inches(12,7)
#    z_lim = ax.get_zlim()
#    ax.view_init(elevation, azimuth) # <- to rotate the plot
#    plt.subplots_adjust(top=1.2, right=1.2, bottom = -0.1)
    plt.show()
  

out_geds = widget.interactive_output(plot_geds_3D, {'zrange': zrange_slider,
                                                    'elevation': elevation_slider,
                                                    'azimuth': azimuth_slider,
                                                    'time_select': geds_time_select,
                                                    'parameter': par_geds_buttons, 
                                                    'string': geds_map_buttons,
                                                    'run': run_button})

form_items = [
    widget.VBox([geds_time_select]),
    widget.VBox([run_button, par_geds_buttons, geds_map_buttons]),            
    widget.VBox([zrange_slider]),
    widget.VBox([elevation_slider, azimuth_slider]),
    out_geds
        ]

widget.VBox(form_items)